In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

In [2]:
data = pd.read_excel('../data/filtered_broker_target.xlsx')

In [3]:
data = data.set_index('Broker').transpose()

In [4]:
data = data.reset_index().rename(columns= {'index':'FiscalYear'})


In [17]:
data['Time'] = pd.to_datetime(data['Time'], format = "%Y")

In [5]:
for name in data.columns[1:]:
    model = ARIMA(data[name], order=(2, 1, 3))
    model_fit = model.fit()
    data[name] = data[name].fillna(model_fit.predict(start=0, end=len(data)-1))


/home/codespace/.python/current/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/home/codespace/.python/current/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/home/codespace/.python/current/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/home/codespace/.python/current/lib/python3.10/site-packages/

In [7]:
data['FiscalYear'] = pd.to_datetime(data['FiscalYear'], format="%Y")

In [9]:
data.to_excel('../data/filtered_filledna.xlsx',index=False)

In [43]:
import pandas as pd
from darts import TimeSeries
from darts.metrics.metrics import rmse
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel, NBEATSModel, RandomForest, NHiTSModel, NaiveMovingAverage, XGBModel, CatBoostModel
from darts.utils.utils import ModelMode

In [26]:
cutoff_year = '2021'

In [22]:
series = TimeSeries.from_dataframe(data, time_col="Time")

In [27]:
train_series, val_series = series.split_before(pd.Timestamp(cutoff_year))

In [46]:
def get_score(name, training_scaled, validation_scaled):

    score = {}

    # baseline 
    moving_average = NaiveMovingAverage(input_chunk_length=3)
    moving_average.fit(training_scaled[name])
    prediction = moving_average.predict(2024 - int(cutoff_year))

    moving_average_score = rmse(validation_scaled[name], prediction)
    score["MovingAverage"] = moving_average_score

    # statistical 
    es = ExponentialSmoothing(trend=ModelMode.ADDITIVE, seasonal=None)
    es.fit(training_scaled[name])
    prediction = es.predict(2024 - int(cutoff_year))


    es_score = rmse(validation_scaled[name], prediction)
    score["ExponentialSmoothing"] = es_score


    # ml
    rf = RandomForest(lags=1, n_estimators=10)
    rf.fit(training_scaled[name])
    prediction = rf.predict(2024 - int(cutoff_year))
    rf_score = rmse(validation_scaled[name], prediction)
    
    score["RandomForest"] = rf_score

    cb = CatBoostModel(lags=1, n_estimators=10)
    cb.fit(training_scaled[name])
    prediction = rf.predict(2024 - int(cutoff_year))
    cb_score = rmse(validation_scaled[name], prediction)
    
    score["CatBoost"] = cb_score

    xgb = XGBModel(lags=1, n_estimators=10)
    xgb.fit(training_scaled[name])
    prediction = xgb.predict(2024 - int(cutoff_year))
    xgb_score = rmse(validation_scaled[name], prediction)
    
    score["XGBModel"] = xgb_score

    # dl

    nhits = NHiTSModel(
    input_chunk_length=3,
    output_chunk_length=4,
    num_blocks=2,
    n_epochs=5,
)
    nhits.fit(training_scaled[name])
    prediction = nhits.predict(2024 - int(cutoff_year))
    nhits_score = rmse(validation_scaled[name], prediction)
    score["NHiTS"] = nhits_score


    return score
    



In [47]:
moving_average_list = []
exponential_smoothing_list = []
rf_list = []
nhits_list = []
xgb_list = []
catboost_list = []
for name in data.columns[1:]:

    scaler = Scaler()
    training_scaled = scaler.fit_transform(train_series[name])
    validation_scaled = scaler.transform(val_series[name])
    scores = get_score(name, training_scaled, validation_scaled)

    moving_average_list.append(scores['MovingAverage'])
    exponential_smoothing_list.append(scores['ExponentialSmoothing'])
    rf_list.append(scores['RandomForest'])
    nhits_list.append(scores['NHiTS'])
    xgb_list.append(scores['XGBModel'])
    catboost_list.append(scores['CatBoost'])
    

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 1.6 M  | train
-------------------------------------------------------------
1.6 M     Trainable params
1.5 K     Non-trainable params
1.6 M     Total params
6.423     Total estimated model params size (MB)


Epoch 4: 100%|██████████| 1/1 [00:00<00:00, 38.55it/s, train_loss=0.164]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 1/1 [00:00<00:00, 36.35it/s, train_loss=0.164]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Epoch 4: 100%|██████████| 1/1 [00:00<00:00, 38.11it/s, train_loss=0.0773]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 1/1 [00:00<00:00, 35.98it/s, train_loss=0.0773]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 148.47it/s]


In [48]:
print(f"the aveage score of moving average is {round(sum(moving_average_list)/len(moving_average_list),3)}")
print(f"the aveage score of exponential Smoothing is {round(sum(exponential_smoothing_list)/len(exponential_smoothing_list),3)}")
print(f"the aveage score of random forest is {round(sum(rf_list)/len(rf_list),3)}")
print(f"the aveage score of nhits is {round(sum(nhits_list)/len(nhits_list),3)}")
print(f"the average score of catboost is {round(sum(catboost_list)/len(catboost_list),3)}")
print(f"the average score of xgboost is {round(sum(xgb_list)/len(xgb_list),3)}")



the aveage score of moving average is 0.512
the aveage score of exponential Smoothing is 0.651
the aveage score of random forest is 0.535
the aveage score of nhits is 0.779
the average score of catboost is 0.535
the average score of xgboost is 0.573
